In [0]:
from google.colab import drive
from os import makedirs
from os import system
from os.path import join
from os.path import exists
from shutil import move
from datetime import datetime as dt
import sys

# Working with a github project on colab in google drive

In [0]:
GIT_REPOSITORY_OWNER = "td-ct"           # GITHUB PROJECT'S HOSTS username
GIT_REPOSITORY = "test"          # GITHUB PROJECT'S HOSTS username

GIT_TOKEN = ""                   # Your personal github token can be generated in your GitHub account (settings->developer settings->personal access tokens)

GDRIVE_ROOT = '/content/drive'   # default for the drive
PROJ = 'My Drive/Colab-GitHub'   # path to your project on Drive

#TD Some Concatenations... Nothing to change here!
PROJECT_PATH = join(GDRIVE_ROOT, PROJ)
REPO_PATH = join(PROJECT_PATH,GIT_REPOSITORY)
REPO_DOT_PATH = join(REPO_PATH,".git/")
GIT_PATH = "https://"+GIT_TOKEN+"@github.com/"+GIT_REPOSITORY_OWNER+"/"+GIT_REPOSITORY+".git"

In [0]:
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    """
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

## Google Drive - Connect to your own drive

In [17]:
#drive.mount(GDRIVE_ROOT)
drive.mount(GDRIVE_ROOT, force_remount=True)

Mounted at /content/drive


## GitHub Initial "clone" into fresh directory - Download your project and prepare everything
You will need your private token for github, which you can generate going to github -> settings -> Developer Settings -> Personal Access Token -> Check "repo" -> generate it -> save it (you won't see it later again, only chance!

In [18]:
#!mkdir "{PROJECT_PATH}"     # in case we haven't created it already   

try: 
  makedirs(PROJECT_PATH)
  print("Colab-Github base project directory '%s' was created successfully on GDrive" %PROJECT_PATH) 
except OSError as error: 
  #print(error)
  try: 
    makedirs(PROJECT_PATH, exist_ok = True)
    print("Colab-Github base project directory '%s' already on GDrive" %PROJECT_PATH) 
  except OSError as error: 
    #print(error)
    print("Fatal: Base Colab-GitHub directory '%s' on GDrive can not be created" %PROJECT_PATH)

gitPull = False

try: 
  makedirs(REPO_PATH)
  print("Directory '%s' was created successfully" %REPO_PATH) 
except OSError as error: 
  try: 
    makedirs(REPO_PATH, exist_ok = True) 


    print("Checking '%s'" % REPO_DOT_PATH)
    if (exists(REPO_DOT_PATH)):
      gitPull = query_yes_no("Update (git pull) existing Repo? Otherwise, we move your local repo to another folder and let \"git clone\" populate a new one.","yes")
    else:
      print("Directory '%s' does not appear to be a valid git repo." % REPO_PATH)
    if not gitPull:
      MOVED_REPO_PATH = REPO_PATH+"_moved_"+dt.now().strftime('%Y%m%d%H%M%S')
      print("Directory '%s' exists. Moving to '%s' sake of safety!" % (REPO_PATH,MOVED_REPO_PATH) )
      move(REPO_PATH, MOVED_REPO_PATH) 
      makedirs(REPO_PATH)
      print("Empty directory '%s' was created successfully" %REPO_PATH) 
  except OSError as error:
    print("Fatal: Directory '%s' can not be created" %REPO_PATH) 

if gitPull:
  !git -C "{REPO_PATH}" fetch --all                       # Get all remote changes
  if (query_yes_no("Loose your local changes and update with remote master branch? Choosing 'n' you might want to use your own git software to manually merge...","no")):
    print("Hard reset your local changes!")
    !git -C "{REPO_PATH}" reset --hard origin/master        # Replace your changes with the remote ones!!!
else:
  !git clone "{GIT_PATH}" "{REPO_PATH}"                   # Pupulate an empty filder with a fresh copy from the remote server

Colab-Github base project directory '/content/drive/My Drive/Colab-GitHub' already on GDrive
Checking '/content/drive/My Drive/Colab-GitHub/test/.git/'
Update (git pull) existing Repo? Otherwise, we move your local repo to another folder and let "git clone" populate a new one. [Y/n] y
Fetching origin
Loose your local changes and update with remote master branch? Choosing 'n' you might want to use your own git software to manually merge... [y/N] y
Hard reset your local changes!
HEAD is now at 023725a I addes a line to test.ipynb


## Colab - Open the code in colab and work on it

## GitHub "push" - Upload your changes to the reposiory and benefit from version tracking

In [20]:
GIT_EMAIL = "timo.damm@rad.uni-kiel.de"                   # replace with yours
GIT_USERNAME = "td-ct"                                        # replace with yours
GIT_BRANCH_NAME = "master"                                # except you want to start your own subbranch

!git config --global user.email "{GIT_EMAIL}"             # Always reset the git config data, as we are on a could system...
!git config --global user.name "{GIT_USERNAME}"               # Always reset the git config data, as we are on a could system...

!git -C "{REPO_PATH}" add .                               # Add all new files to to repo, if they are not ignored due to .gitigonre settings!

GIT_COMMIT_MESSAGE = "Commit Message: [New Feature.../Bugfix on...???]"
sys.stdout.write(GIT_COMMIT_MESSAGE)
GIT_COMMIT_MESSAGE = input()

!git -C "{REPO_PATH}" commit -m "{GIT_COMMIT_MESSAGE}"    # Commit your changes with a description locally
!git -C "{REPO_PATH}" push origin master                  # Push your changes to the remote GitHub repo!

Commit Message: [New Feature.../Bugfix on...???]Feierabend!
[master bb8c514] Feierabend!
 1 file changed, 2 insertions(+)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 319 bytes | 29.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://github.com/td-ct/test.git
   023725a..bb8c514  master -> master


## Finished - Your work is saved to GitHub